In [3]:
import os
from pandas import read_csv, DataFrame
import numpy as np
import pandas as pd
import pyedra
import matplotlib.pyplot as plt

path = os.getcwd()


###  Catálogo completo

Se aplica el código Pyedra para determinar los parámetros de la función de fase a partir de los datos de fase (alpha) y la magnitud V de Johnson reducida (v) calculada anteriormente para cada catálogo.

Luego se determina la magnitud V (V_fit) a partir de los valores de H y G obtenidos (H_fit y G_fit).

Para aplicar el código se deben tener mínimo 2 observaciones.

In [5]:
df = read_csv(path+"/Datos/Jupiter_trojans.csv")

In [8]:
### Eliminar asteroides con menos de dos observaciones
index = np.array(np.where(df.id.duplicated(keep=False)))
df = df.iloc[index[0]]

### Implementar código Pyedra
HG_all = pyedra.HG_fit(df, idc="id", alphac="alpha", magc="v")


##Rellenar tabla original con H, G (con error porcentual) y V ajustados

# errores porcentuales
HG_all.loc[:,'%error_H'] = np.abs(HG_all.loc[:,'error_H']/HG_all.loc[:,'H']*100)
HG_all.loc[:,'%error_G'] = np.abs(HG_all.loc[:,'error_G']/HG_all.loc[:,'G']*100)

# H y G (con %error)
HG_all.rename(columns={'H': 'H_fit','G':'G_fit','R':'r'}, inplace=True)
df = pd.merge(df, HG_all, on=('id'), how='outer')

# V  
V = df.loc[:,'v']
Phase = df.loc[:,'alpha']
H = df.loc[:,'H_fit']
G = df.loc[:,'G_fit']

def V_fit(alpha, H, G):                                        
    x = alpha * np.pi / 180
    y = H - 2.5 * np.log10(
        (1 - G) * np.exp(-3.33 * np.tan(x / 2) ** 0.63)
        + G * np.exp(-1.87 * np.tan(x / 2) ** 1.22))
    return y
df['V_fit'] = V_fit(Phase, H, G)

display(df)

c:\users\saruk\pyedra-master\pyedra\hg_model.py:243: RuntimeWarning: invalid value encountered in log10
  H = -2.5 * np.log10(a + b)
c:\users\saruk\pyedra-master\pyedra\hg_model.py:243: RuntimeWarning: invalid value encountered in log10
  H = -2.5 * np.log10(a + b)
C:\Users\saruk\anaconda3\lib\site-packages\scipy\optimize\minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
c:\users\saruk\pyedra-master\pyedra\hg_model.py:243: RuntimeWarning: invalid value encountered in log10
  H = -2.5 * np.log10(a + b)
c:\users\saruk\pyedra-master\pyedra\hg_model.py:243: RuntimeWarning: invalid value encountered in log10
  H = -2.5 * np.log10(a + b)
C:\Users\saruk\anaconda3\lib\site-packages\scipy\optimize\minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
c:\users\saruk\pyedra-master\pyedra\hg_m

C:\Users\saruk\anaconda3\lib\site-packages\scipy\optimize\minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\saruk\anaconda3\lib\site-packages\scipy\optimize\minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\saruk\anaconda3\lib\site-packages\scipy\optimize\minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\saruk\anaconda3\lib\site-packages\scipy\optimize\minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\saruk\anaconda3\lib\site-packages\scipy\optimize\minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
 

C:\Users\saruk\anaconda3\lib\site-packages\scipy\optimize\minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\saruk\anaconda3\lib\site-packages\scipy\optimize\minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
c:\users\saruk\pyedra-master\pyedra\hg_model.py:243: RuntimeWarning: invalid value encountered in log10
  H = -2.5 * np.log10(a + b)
c:\users\saruk\pyedra-master\pyedra\hg_model.py:243: RuntimeWarning: invalid value encountered in log10
  H = -2.5 * np.log10(a + b)
C:\Users\saruk\anaconda3\lib\site-packages\scipy\optimize\minpack.py:828: OptimizeWarning: Covariance of the parameters could not be estimated
  warnings.warn('Covariance of the parameters could not be estimated',
C:\Users\saruk\anaconda3\lib\site-packages\scipy\optimize\minpack.py:828: OptimizeWarning: C

,Name,id,jd,a_orb,e_orb,i_orb,Delta,R,alpha,umag,...,G,H_fit,error_H,G_fit,error_G,r,observations,%error_H,%error_G,V_fit
0,NaN,588,2.456893e+06,5.209202,0.147923,10.319917,5.398240,6.064449,7.7575,NaN,...,0.15,7.942679,0.961282,-0.279241,0.595024,0.196752,18,12.102746,213.086475,8.849012
1,NaN,588,2.456892e+06,5.209202,0.147923,10.319917,5.397643,6.057591,7.8178,NaN,...,0.15,7.942679,0.961282,-0.279241,0.595024,0.196752,18,12.102746,213.086475,8.853787
2,NaN,588,2.456892e+06,5.209202,0.147923,10.319917,5.397720,6.058477,7.8101,NaN,...,0.15,7.942679,0.961282,-0.279241,0.595024,0.196752,18,12.102746,213.086475,8.853178
3,NaN,588,2.456892e+06,5.209202,0.147923,10.319917,5.397903,6.060584,7.7916,NaN,...,0.15,7.942679,0.961282,-0.279241,0.595024,0.196752,18,12.102746,213.086475,8.851714
4,NaN,588,2.456893e+06,5.209202,0.147923,10.319917,5.397980,6.061467,7.7839,NaN,...,0.15,7.942679,0.961282,-0.279241,0.595024,0.196752,18,12.102746,213.086475,8.851104
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8187,2013 CB178,390353,2.451961e+06,5.133892,0.111926,30.096112,4.137121,5.068220,4.1637,22.05,...,0.15,13.873063,inf,0.783805,inf,1.000000,2,inf,inf,13.985499
8188,2013 AD70,393059,2.452731e+06,5.165245,0.093502,12.770055,4.381753,5.354184,2.7225,22.43,...,0.15,12.408961,inf,-2.152225,inf,1.000000,2,inf,inf,14.222127
8189,2013 AD70,393059,2.453527e+06,5.165245,0.093502,12.770055,4.673314,5.666940,2.2902,23.07,...,0.15,12.408961,inf,-2.152225,inf,1.000000,2,inf,inf,13.885999
8190,2013 TZ46,412394,2.451815e+06,5.149318,0.063366,27.263493,4.081393,4.950360,6.3638,22.59,...,0.15,13.494623,inf,-0.691843,inf,1.000000,2,inf,inf,14.692241


### Troyanos comunes 